#### The Python notebook with the Data Cleaning, Model initailization, & Model fine-tuning code

### - Install Needed Libraries

In [1]:
! pip install -U accelerate --quiet
! pip install --upgrade accelerate
! pip install -U git+https://github.com/huggingface/transformers.git
! pip install -U git+https://github.com/huggingface/accelerate.git
! git clone https://github.com/aub-mind/arabert.git
! pip install evaluate
! pip install gradio

^C
ERROR: Operation cancelled by user


UnboundLocalError: local variable 'child' referenced before assignment

### Import Needed libraries

In [ ]:
import pandas as pd
import numpy as np
import re
from arabert.aragpt2.grover.modeling_gpt2 import GPT2LMHeadModel
from torch.utils.data import Dataset, DataLoader
import torch
import evaluate
from evaluate import logging
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from transformers import AutoTokenizer
from transformers import TextDataset,DataCollatorForLanguageModeling
from transformers import GPT2TokenizerFast, pipeline
from transformers import  AutoModel, utils
# from transformers import BertTokenizer,BertForQuestionAnswering
from transformers import  Trainer, TrainingArguments,AutoModelWithLMHead, GPT2LMHeadModel

### - Read Data 

In [ ]:
df = pd.read_csv('/kaggle/input/fatawa/88kData.csv')
df.head(10)
# No. of records
len(df)

### - Cleaning data

In [ ]:
#remove ID column
df = df.drop('ID', axis=1)
# remove Null value
df = df.dropna()

In [ ]:
NewList = []

In [ ]:
# append unneeded fatwa

NewList+=list(df[df['ans'].str.contains('انظر إلى الفتوى')].index)
NewList+=list(df[df['ans'].str.contains('رقم:')].index)
NewList+=list(df[df['ans'].str.contains('فتوى رقم')].index)

In [ ]:
# drop unneeded fatwa
df = df.drop(index=NewList, axis=0)

In [ ]:
df[df['ans'].str.contains('นู')]

In [ ]:
df = df.drop(index=84488, axis=0)

In [ ]:
# No. of new records
len(df)

In [ ]:
def extract_arabic_and_english_numerics(text):
    # remove english letter
    arabic_pattern = r'[^a-zA-Z\s0-9]+'
    numeric_pattern = r'\d+'
    combined_pattern = f'({arabic_pattern}|{numeric_pattern})'
    
    matches = re.findall(combined_pattern, text)    
    extracted_text = ' '.join(matches)
    
    return extracted_text

In [ ]:
#Applying the function on the ['ans'] column
df['ans'] = df['ans'].apply(extract_arabic_and_english_numerics)
df['ans']

In [ ]:
#Applying the function on the ['ques'] column
df['ques'] = df['ques'].apply(extract_arabic_and_english_numerics)
df['ques']

In [ ]:
#Applying the function on the ['title'] column
df['title'] = df['title'].apply(extract_arabic_and_english_numerics)
df['title']

In [ ]:
newlist2 = []

In [ ]:
#Visualize the answers with length less than 118 charachter
for k in range(len(df)):
    
    if len(df['ans'].iloc[k]) < 118:
        print(df['ans'].iloc[k])

In [ ]:
#Dropping every row that contains an answer with length less than 118 charachter
for k in range(len(df)):
    
    if len(df['ans'].iloc[k]) < 118:
        newlist2.append(k)

In [ ]:
# remove from data
newlist2 = list(df.iloc[newlist2].index)
df = df.drop(index=newlist2, axis=0)

In [ ]:
#Making all the dataset columns as a string, and dropping duplicates from the dataset.
df = df.astype(str)
df.drop_duplicates(inplace=True)

In [ ]:
# resize data to 20000
df = df[0:20000]

In [ ]:
#Save the cleaned version of the data to the Machine as .csv file
df.to_csv("Finalized_Data2.csv",index=False)

In [ ]:
df = pd.read_csv("Finalized_Data2.csv")

### - Model Intialization & Data Adjustement for Model Fine-Tuning

In [ ]:
df["full_text"]=" question: " + df["title"]+df["ques"]+" answer: "+df["ans"]

In [ ]:
df["full_text"][1]

In [ ]:
# split the data to train and test

from sklearn.model_selection import train_test_split
train, test = train_test_split(df["full_text"],test_size=0.10, shuffle=True, random_state=42)

In [ ]:
# save train and test data 

train.to_csv(r'train_dataset1.txt', header=None, index=None, sep=' ', mode='a')
test.to_csv(r'test_dataset1.txt', header=None, index=None, sep=' ', mode='a')

### Pretrain aragpt2 model

In [ ]:
!pip install sacrebleu

In [ ]:
from sacrebleu import corpus_bleu
import numpy as np

def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)

    # Set pad token
    if tokenizer.pad_token_id is not None:
        labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    else:
        labels_ids[labels_ids == -100] = 0

    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    bleu_score = round(corpus_bleu(pred_str, [label_str]).score, 4)

    return {"bleu": bleu_score}


In [ ]:
model = AutoModelWithLMHead.from_pretrained("aubmindlab/aragpt2-base")
tokenizer = AutoTokenizer.from_pretrained("aubmindlab/aragpt2-base")
tokenizer.add_tokens(['<question>', '<answer>'])
train_path = 'train_dataset1.txt'
test_path = 'test_dataset1.txt'

In [ ]:
model.config.max_length=100

### - (ARAGPT2) Fine-Tuning

In [ ]:
def load_dataset(train_path, test_path, tokenizer):
    train_dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=train_path,
        block_size=128
    )

    test_dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=test_path,
        block_size=128
    )

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False
    )
    
    eval_data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False
    )

    return train_dataset, test_dataset, data_collator, eval_data_collator

train_dataset, test_dataset, data_collator, eval_data_collator = load_dataset(train_path, test_path, tokenizer)


In [ ]:
eval_data_collator

In [ ]:
len(train_dataset)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# clean memory
torch.cuda.empty_cache()

In [ ]:
batch_size=8

In [ ]:
# !pip install --upgrade transformers

In [ ]:
from transformers import  Seq2SeqTrainingArguments
from transformers import Seq2SeqTrainer

In [ ]:
# set training arguments - these params are not really tuned, feel free to change
training_args = Seq2SeqTrainingArguments(
    output_dir="./bert2gpt-Training",
    evaluation_strategy="epoch",
    save_strategy = "epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    num_train_epochs=2,
    do_train=True,
    do_eval=True,
    fp16=True,
    overwrite_output_dir=True,
    learning_rate = 1e-5,
    weight_decay=0.01,
    warmup_ratio = 0.05,
    seed = 1995,
    save_total_limit = 2,
    load_best_model_at_end = True,
    #push_to_hub=True,
    optim = "adamw_hf",
)
#

# instantiate trainer
trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    compute_metrics=compute_metrics,
    data_collator = data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset

)
#    save_total_limit=3,
#

In [ ]:
trainer.train()

In [ ]:
print(trainer.state.log_history)

In [ ]:
import numpy as np

# حساب خسارة التدريب وخسارة التحقق
train_loss = np.mean(trainer.state.log_history[1]["train_loss"])
validation_loss = np.mean(trainer.state.log_history[0]["eval_loss"])

# عرض النتائج
print("Training Loss:", train_loss)
print("Validation Loss:", validation_loss)

In [ ]:
import matplotlib.pyplot as plt

train_loss_values = [log["train_loss"] for log in trainer.state.log_history[1:]]
validation_loss_values = [log["eval_loss"] for log in trainer.state.log_history[0:]]

epochs = range(1, len(train_loss_values) + 1)

plt.plot(epochs, train_loss_values, label='Train Loss')
plt.plot(epochs, validation_loss_values, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("aubmindlab/aragpt2-base")
tokenizer = AutoTokenizer.from_pretrained("aubmindlab/aragpt2-base")

In [ ]:
# حساب المقاييس على مجموعة الاختبار
eval_results = trainer.evaluate()

# عرض نتائج المقاييس
print("Meteor Score:", eval_results['eval_meteor_score'])
print("ROUGE2 Precision:", eval_results['eval_rouge2_precision'])
print("ROUGE2 Recall:", eval_results['eval_rouge2_recall'])
print("ROUGE2 F-Measure:", eval_results['eval_rouge2_f_measure'])

In [ ]:
import gradio as gr
from transformers import  pipeline

generation_pipeline = pipeline('text-generation',model="/kaggle/input/islamweb", tokenizer='aubmindlab/aragpt2-base')


def prediction(str):

    str=generation_pipeline(str,max_length=300)[0]['generated_text']
    start_index = str.index('answer:')
    if 'والله أعلم' in str:
        stop_index = str.index('والله أعلم')
        str = str[start_index+8:stop_index+11].strip()
        
    else:
        str = str[start_index+8:].strip()

    return str

demo = gr.Interface(fn=prediction,outputs=gr.Textbox(label="الاجابه"),inputs=gr.Textbox(label="ما هو سؤالك؟"),title="نَوَّرْ",css="body {background-color: green,}")

demo.launch(share=True)

In [ ]:
trainer.save_model()
trainer.save_state()

In [ ]:
//# لحد هنا الموديل 